# Neural Network

In [ ]:
import os

import numpy as np

from matplotlib import pyplot

from scipy import optimize

from scipy.io import loadmat

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive")

## Data

In [ ]:
input_layer_size = 400

num_labels = 10

data = loadmat("/content/MyDrive/MyDrive/Machine Learning Data/EX_3/ex3data1.mat")

X,y = data["X"], data["y"].ravel()

y[y==10] = 0

m = y.size

In [ ]:
print(X.shape)
print(m)

## Visualize Data

In [ ]:
def displayData(X, example_width=None, figsize=(10, 10)):

    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  
    else:
        raise IndexError('Input X should be 1 or 2 dimensional.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

In [ ]:
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

displayData(sel)

In [ ]:
# test values for the parameters theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# test values for the inputs
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# test values for the labels
y_t = np.array([1, 0, 1, 0, 1])

# test value for the regularization parameter
lambda_t = 3

## Sigmoid

In [ ]:
def sigmoid(z):
  z = np.array(z)
  g = np.zeros(z.shape)

  g = 1/(1 + np.exp(-z))

  return g

## Cost_Function 

In [ ]:
def cost_func(theta,X,y,lambda_):
  
  m = y.size

  if y.dtype == bool:

    y = y.astype(int)

  J = 0
  grad = np.zeros(theta.shape)

  h = sigmoid(np.dot(X,theta))

  temp = theta
  temp[0] = 0
  J = (1/(m)) * np.sum(-y.dot(np.log(h)) - (1-y).dot(np.log(1-h)))+ (lambda_/(2*m))* np.sum(np.square(temp))

  grad = (1/m) * (h-y).dot(X)

  grad = grad + (lambda_/m) * (temp)
 
  return J, grad


In [ ]:
J, grad = cost_func(theta_t, X_t, y_t, lambda_t)

print('Cost         : {:.6f}'.format(J))
print('Expected cost: 2.534819')
print('-----------------------')
print('Gradients:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Expected gradients:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

## OneVsAll theta

In [ ]:
def oneVsAll(X,y,num_labels,lambda_):
  m,n = X.shape

  all_theta = np.zeros((num_labels,n+1))

  X = np.concatenate([np.ones((m,1)),X],axis = 1)

  for c in np.arange(num_labels):

    initial_theta = np.zeros(n+1)

    options = {"maxiter": 50}

    res = optimize.minimize(cost_func,
                          initial_theta,
                          (X, y==c ,lambda_),
                          jac=True,
                          method='CG',
                          options=options)

    all_theta[c] = res.x

  return all_theta

In [ ]:
lambda_ = 0.1

In [ ]:
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

## PredictOneVsAll 

In [ ]:
def predictOneVsAll(all_theta, X):
  m = X.shape[0]

  num_labels = all_theta.shape[0]

  p = np.zeros(m)

  X = np.concatenate([np.ones((m,1)),X],axis = 1)

  p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

  return p

In [ ]:
pred = predictOneVsAll(all_theta, X)
print('Training Set Accuracy: {:.2f}%'.format(np.mean(pred == y) * 100))

# Neural Networks 

In [ ]:
data = loadmat("/content/MyDrive/MyDrive/Machine Learning Data/EX_3/ex3data1.mat")

X, y = data["X"], data["y"].ravel()

y[y==10] = 0

m = y.size

indices = np.random.permutation(m)

rand_indices = np.random.choice(m, 100, replace = False)

sel = X[rand_indices,:]

displayData(sel)

In [ ]:
initial_layer_size = 400
hidden_layer_size = 25
num_labels = 10

weights = loadmat("/content/MyDrive/MyDrive/Machine Learning Data/EX_3/ex3weights.mat")

Theta1 , Theta2 = weights["Theta1"] , weights["Theta2"]

Theta2 = np.roll(Theta2, 1, axis = 0)


In [ ]:
print(Theta1.shape,Theta2.shape)

In [ ]:
def predict(Theta1, Theta2, X):
  if X.ndim == 1:
    X = X[None]

  m = X.shape[0]
  num_labels = Theta2.shape[0]

  p = np.zeros(X.shape[0])

  X = np.concatenate([np.ones((m,1)),X],axis = 1)

  a2 = sigmoid(X.dot(Theta1.T))

  a2 = np.concatenate([np.ones((a2.shape[0],1)),a2],axis = 1)

  p = np.argmax(sigmoid(a2.dot(Theta2.T)),axis = 1)

  return p 

In [ ]:
pred = predict(Theta1, Theta2, X)
print('Training Set Accuracy: {:.1f}%'.format(np.mean(pred == y) * 100))

In [ ]:
if indices.size > 0:
    i, indices = indices[0], indices[1:]
    displayData(X[i, :], figsize=(4, 4))
    pred = predict(Theta1, Theta2, X[i, :])
    print('Neural Network Prediction: {}'.format(*pred))
else:
    print('No more images to display!')